In [1]:
import tensorflow as tf
import tensorflow.keras

import keras
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam, SGD

import matplotlib.pyplot as plt

import numpy as np
from numpy import array

import pandas as pd

import csv
import itertools

import time

Using TensorFlow backend.


# Import expression status of WBP2

In [2]:
wbp2 = pd.read_csv('wbp2stat.csv')
wbp2= wbp2.values.tolist()
print(len(wbp2))

112


# Import expression data

In [3]:
top1000 = pd.read_csv('top1000exp.csv')
top1000 = top1000.transpose()
top1000 = top1000.values.tolist()
bot1000 = pd.read_csv('bot1000exp.csv')
bot1000 = bot1000.transpose()
bot1000 = bot1000.values.tolist()
rand1000 = pd.read_csv('rand1000exp.csv')
rand1000 = rand1000.transpose()
rand1000 = rand1000.values.tolist()

# Correct for class imablance with synthetic data

In [4]:
x = array(wbp2)
class_diff = np.count_nonzero(x == 0) - np.count_nonzero(x == 1)

#Extracting WBP2 upregulation samples
idx = np.where(x == 1)[0].tolist()
for j in range(0,3):
    if j == 0:
        TN = top1000
    elif j == 1:
        TN = bot1000
    else:
        TN = rand1000
    wbp2upTN = []
    for i in idx:
        wbp2upTN.append(TN[i])
    idx = np.random.permutation(len(wbp2upTN))
    wbp2upTN = [wbp2upTN[i] for i in idx]
    wbp2upTN = wbp2upTN [0:class_diff]
    #Adding noise to create synthetic samples
    noise = np.random.normal(1,0.1,len(wbp2upTN[0]))
    noiseTN = wbp2upTN * noise
    #Adding the synthetic sample to the dataset
    TN = list(itertools.chain(TN, noiseTN))
    if j == 0:
        top1000 = TN
    elif j == 1:
        bot1000 = TN
    else:
        rand1000 = TN

for i in range(1,class_diff):
    wbp2.append([1]) 

# Increasing sample size with synthetic data

In [5]:
for x in range(0,10):
    wbp2 = wbp2 + wbp2
    #Generate synthetic data for top1000
    noise = np.random.normal(1,0.1,len(top1000[0]))
    noiseTN = top1000 * noise
    top1000 = list(itertools.chain(top1000, noiseTN))
    #enerate synthetic data for bot1000
    noise = np.random.normal(1,0.1,len(bot1000[0]))
    noiseTN = bot1000 * noise
    bot1000 = list(itertools.chain(bot1000, noiseTN))
    #enerate synthetic data for rand1000
    noise = np.random.normal(1,0.1,len(rand1000[0]))
    noiseTN = rand1000 * noise
    rand1000 = list(itertools.chain(rand1000, noiseTN))
    print(len(wbp2))
print('Done')

298
596
1192
2384
4768
9536
19072
38144
76288
152576
Done


In [6]:
top1000acc = []
bot1000acc = []
rand1000acc = []

In [7]:
# Training Parameters
learning_rate = 0.001 
training_epochs = 15
batch_size = 1000

opt = SGD(lr = learning_rate)

# Network Parameters
n_hidden_1 = 40 # 1st layer number of neurons
n_hidden_2 = 20 # 2nd layer number of neurons
n_hidden_3 = 10 # 2nd layer number of neurons

n = 400 #number of loop

for k in range(0,n):
    t = time.time()
    
    #Random shuffle of dataset
    numsample = len(wbp2)
    idx = np.random.permutation(numsample) 
    wbp2 = [wbp2[i] for i in idx]
    top1000 = [top1000[i] for i in idx]
    bot1000 = [bot1000[i] for i in idx]
    rand1000 = [rand1000[i] for i in idx]
    
    #Splitting of dataset into training and validation set
    percent_train = 0.7
    #Top1000
    num_train = int(percent_train * numsample)
    train_top_y = array(wbp2[0:num_train])
    train_top_x = array(top1000[0:num_train])
    test_top_y  = array(wbp2[num_train:numsample])
    test_top_x  = array(top1000[num_train:numsample])
    #Bot1000
    train_bot_y = array(wbp2[0:num_train])
    train_bot_x = array(bot1000[0:num_train])
    test_bot_y  = array(wbp2[num_train:numsample])
    test_bot_x  = array(bot1000[num_train:numsample])
    #Rand1000
    train_rand_y = array(wbp2[0:num_train])
    train_rand_x = array(rand1000[0:num_train])
    test_rand_y  = array(wbp2[num_train:numsample])
    test_rand_x  = array(rand1000[num_train:numsample])

    # convert class vectors to binary One Hot Encoded
    n_classes = 2
    train_top_y = keras.utils.to_categorical(train_top_y, n_classes)
    test_top_y = keras.utils.to_categorical(test_top_y, n_classes)
    train_bot_y = keras.utils.to_categorical(train_bot_y, n_classes)
    test_bot_y = keras.utils.to_categorical(test_bot_y, n_classes)
    train_rand_y = keras.utils.to_categorical(train_rand_y, n_classes)
    test_rand_y = keras.utils.to_categorical(test_rand_y, n_classes)
    
    n_input_top =  len(train_top_x[0])
    n_input_bot =  len(train_bot_x[0])
    n_input_rand =  len(train_rand_x[0])
    
    Inp_top = Input(shape=(n_input_top,))
    x = Dense(n_hidden_1, activation='relu', name = "Dense_1")(Inp_top)
    x = Dropout(0.4, name = "Dropout_01")(x)
    x = Dense(n_hidden_2, activation='relu', name = "Dense_2")(x)
    x = Dropout(0.4, name = "Dropout_02")(x)
    x = Dense(n_hidden_3, activation='relu', name = "Dense_3")(x)
    output_top = Dense(n_classes, activation='softmax', name = "Outputlayer")(x)
    model_top = Model(Inp_top, output_top)
    model_top.compile(loss='binary_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])
    history_top = model_top.fit(train_top_x, train_top_y,
                        batch_size=batch_size,
                        epochs=training_epochs,
                        verbose=1, # This is for what we want it to display out as it trains 
                        validation_data=(test_top_x, test_top_y))
    
    
    Inp_bot = Input(shape=(n_input_bot,))
    x = Dense(n_hidden_1, activation='relu', name = "Dense_1")(Inp_bot)
    x = Dropout(0.4, name = "Dropout_01")(x)
    x = Dense(n_hidden_2, activation='relu', name = "Dense_2")(x)
    x = Dropout(0.4, name = "Dropout_02")(x)
    x = Dense(n_hidden_3, activation='relu', name = "Dense_3")(x)
    output_bot = Dense(n_classes, activation='softmax', name = "Outputlayer")(x)
    model_bot = Model(Inp_bot, output_bot)
    model_bot.compile(loss='binary_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])
    history_bot = model_bot.fit(train_bot_x, train_bot_y,
                        batch_size=batch_size,
                        epochs=training_epochs,
                        verbose=1, # This is for what we want it to display out as it trains 
                        validation_data=(test_bot_x, test_bot_y))
    
    
    Inp_rand = Input(shape=(n_input_rand,))
    x = Dense(n_hidden_1, activation='relu', name = "Dense_1")(Inp_rand)
    x = Dropout(0.4, name = "Dropout_01")(x)
    x = Dense(n_hidden_2, activation='relu', name = "Dense_2")(x)
    x = Dropout(0.4, name = "Dropout_02")(x)
    x = Dense(n_hidden_3, activation='relu', name = "Dense_3")(x)
    output_rand = Dense(n_classes, activation='softmax', name = "Outputlayer")(x)
    model_rand = Model(Inp_rand, output_rand)
    model_rand.compile(loss='binary_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])
    history_rand = model_rand.fit(train_rand_x, train_rand_y,
                        batch_size=batch_size,
                        epochs=training_epochs,
                        verbose=1, # This is for what we want it to display out as it trains 
                        validation_data=(test_rand_x, test_rand_y))
    
    top_temp = history_top.history['val_acc']
    top1000acc.append(top_temp)
    bot_temp = history_bot.history['val_acc']
    bot1000acc.append(bot_temp)
    rand_temp = history_rand.history['val_acc']
    rand1000acc.append(rand_temp)
    if k%10 == 0:  #update csv every 10 loop
        pd.DataFrame(top1000acc).to_csv("top1000_revised.csv")
        pd.DataFrame(bot1000acc).to_csv("bot1000_revised.csv")
        pd.DataFrame(rand1000acc).to_csv("rand1000_revised.csv")
    elapsed = time.time() - t
    print('Current Loop')
    print(k + 1)
    print('Time taken for this loop')
    print(elapsed)
    print('Time left')
    print(elapsed*(n-k-1))

#Final save to file
pd.DataFrame(top1000acc).to_csv("top1000_revised.csv")
pd.DataFrame(bot1000acc).to_csv("bot1000_revised.csv")
pd.DataFrame(rand1000acc).to_csv("rand1000_revised.csv")

Train on 106803 samples, validate on 45773 samples
Epoch 1/15
106803/106803 [==============================] - 2s 22us/step - loss: 1.5883 - acc: 0.5288 - val_loss: 0.5598 - val_acc: 0.6420
Epoch 2/15
106803/106803 [==============================] - 2s 17us/step - loss: 0.7867 - acc: 0.5826 - val_loss: 0.5355 - val_acc: 0.7185
Epoch 3/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.6938 - acc: 0.6186 - val_loss: 0.5134 - val_acc: 0.7608
Epoch 4/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.6412 - acc: 0.6362 - val_loss: 0.4983 - val_acc: 0.7969
Epoch 5/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.6107 - acc: 0.6531 - val_loss: 0.4946 - val_acc: 0.8040
Epoch 6/15
106803/106803 [==============================] - 2s 17us/step - loss: 0.5874 - acc: 0.6754 - val_loss: 0.4765 - val_acc: 0.8338
Epoch 7/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.5673 - acc: 0.6884 - val_loss: 0

Epoch 13/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.4139 - acc: 0.7964 - val_loss: 0.2799 - val_acc: 0.9543
Epoch 14/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.4071 - acc: 0.8013 - val_loss: 0.2880 - val_acc: 0.9577
Epoch 15/15
106803/106803 [==============================] - 2s 18us/step - loss: 0.3920 - acc: 0.8096 - val_loss: 0.2523 - val_acc: 0.9576
Train on 106803 samples, validate on 45773 samples
Epoch 1/15
106803/106803 [==============================] - 2s 20us/step - loss: 1.7107 - acc: 0.5449 - val_loss: 0.8280 - val_acc: 0.6889
Epoch 2/15
106803/106803 [==============================] - 2s 15us/step - loss: 1.0846 - acc: 0.5694 - val_loss: 0.7611 - val_acc: 0.7149
Epoch 3/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.9187 - acc: 0.5852 - val_loss: 0.6994 - val_acc: 0.7285
Epoch 4/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.8242 - acc: 0.6015 - val_loss

Epoch 10/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.5969 - acc: 0.6238 - val_loss: 0.5266 - val_acc: 0.7272
Epoch 11/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.5875 - acc: 0.6319 - val_loss: 0.5183 - val_acc: 0.7375
Epoch 12/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.5817 - acc: 0.6409 - val_loss: 0.5114 - val_acc: 0.7517
Epoch 13/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.5720 - acc: 0.6509 - val_loss: 0.5056 - val_acc: 0.7667
Epoch 14/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.5661 - acc: 0.6579 - val_loss: 0.4984 - val_acc: 0.7732
Epoch 15/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.5591 - acc: 0.6639 - val_loss: 0.4918 - val_acc: 0.7682
Train on 106803 samples, validate on 45773 samples
Epoch 1/15
106803/106803 [==============================] - 2s 22us/step - loss: 1.7209 - acc: 0.5175 - val_l

106803/106803 [==============================] - 2s 16us/step - loss: 0.8075 - acc: 0.5892 - val_loss: 0.5611 - val_acc: 0.7042
Epoch 3/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.6803 - acc: 0.6331 - val_loss: 0.5238 - val_acc: 0.8028
Epoch 4/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.6169 - acc: 0.6719 - val_loss: 0.4947 - val_acc: 0.8258
Epoch 5/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.5820 - acc: 0.6912 - val_loss: 0.4695 - val_acc: 0.8493
Epoch 6/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.5536 - acc: 0.7107 - val_loss: 0.4484 - val_acc: 0.8676
Epoch 7/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.5308 - acc: 0.7225 - val_loss: 0.4315 - val_acc: 0.8772
Epoch 8/15
106803/106803 [==============================] - 2s 16us/step - loss: 0.5085 - acc: 0.7396 - val_loss: 0.4099 - val_acc: 0.8663
Epoch 9/15
106803/106803 [============

Epoch 11/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.6480 - acc: 0.5769 - val_loss: 0.6296 - val_acc: 0.6681
Epoch 12/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.6429 - acc: 0.5850 - val_loss: 0.6219 - val_acc: 0.6763
Epoch 13/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.6388 - acc: 0.5875 - val_loss: 0.6157 - val_acc: 0.6770
Epoch 14/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.6344 - acc: 0.5931 - val_loss: 0.6090 - val_acc: 0.6824
Epoch 15/15
106803/106803 [==============================] - 2s 15us/step - loss: 0.6278 - acc: 0.6028 - val_loss: 0.6018 - val_acc: 0.6964
Train on 106803 samples, validate on 45773 samples
Epoch 1/15
106803/106803 [==============================] - 3s 26us/step - loss: 1.4701 - acc: 0.5213 - val_loss: 0.8032 - val_acc: 0.5706
Epoch 2/15
106803/106803 [==============================] - 2s 17us/step - loss: 1.0305 - acc: 0.5383 - val_lo

In [8]:
pd.DataFrame(top1000acc).to_csv("top1000_revised.csv")
pd.DataFrame(bot1000acc).to_csv("bot1000_revised.csv")
pd.DataFrame(rand1000acc).to_csv("rand1000_revised.csv")